In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("features/Features10too200hz.csv")
#df = pd.read_csv("features/unfilteredFeatures.csv")

#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.034982,0.033315,0.047836,117.153433,7.663511,0.002288,131.601251,117.153433,0.039296,1239,...,0.043452,1150,1395,2,94.594595,111.041462,1,1001,10010001,audible
1,0.028157,0.026247,0.040353,63.465079,3.670272,0.001628,71.151924,63.465079,0.031567,861,...,0.032478,749,930,1,100.446429,111.273560,1,1001,10010002,audible
2,0.035103,0.034455,0.049304,142.411421,9.862262,0.002431,160.962810,142.411421,0.039675,1552,...,0.050138,1382,1683,7,95.049505,109.108856,1,1001,10010003,audible
3,0.032311,0.032380,0.044599,134.576651,8.284614,0.001989,151.073407,134.576651,0.036272,1551,...,0.054896,1449,1761,9,91.807229,109.532973,1,1001,10010004,audible
4,0.030766,0.030669,0.044142,144.633076,9.159818,0.001948,161.532869,144.633076,0.034361,1743,...,0.050953,1611,1922,6,97.867804,112.357899,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.024542,0.023903,0.041126,57.330769,3.950974,0.001691,49.749426,57.330769,0.021297,674,...,0.005497,710,948,0,71.120690,99.526715,8,8019,80190246,whisper
4796,0.016662,0.016446,0.024829,31.741925,1.174369,0.000616,28.414596,31.741925,0.014916,542,...,0.004438,577,787,0,73.015873,99.631579,8,8019,80190247,whisper
4797,0.019216,0.018149,0.027342,43.312481,1.685064,0.000748,38.049582,43.312481,0.016881,634,...,0.004820,704,953,0,85.714286,107.323391,8,8019,80190248,whisper
4798,0.015547,0.014571,0.022414,26.616147,0.860052,0.000502,24.287830,26.616147,0.014187,521,...,0.004274,522,712,0,72.352941,99.541204,8,8019,80190249,whisper


In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [17]:
def lda_user1_session1():
    train_ids = [1002,1003]
    test_ids = [1001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
  # print("Accuracy for session 1 is "+str(score))
    return score
    
def lda_user1_session2():
    train_ids = [1001,1003]
    test_ids = [1002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
  # print("Accuracy for session 2 is "+str(score))
    return score
    
def lda_user1_session3():
    train_ids = [1001,1002]
    test_ids = [1003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 3 is "+str(score))
    return score
    
def lda_user2_session1():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 1 is "+str(score))
    return score  
    
def lda_user2_session3():
    train_ids = [2001, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 3 is "+str(score))
    return score 
    
def lda_user2_session4():
    train_ids = [2003, 2001, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2004]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
   #print("Accuracy for session 4 is "+str(score))
    return score    
    
def lda_user2_session5():
    train_ids = [2003, 2004, 2001, 2006, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2005]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
   #print("Accuracy for session 5 is "+str(score))
    return score  
    
def lda_user2_session6():
    train_ids = [2003, 2004, 2005, 2001, 2007, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2006]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
   #print("Accuracy for session 6 is "+str(score))
    return score   
    
def lda_user2_session7():
    train_ids = [2003, 2004, 2005, 2006, 2001, 2008, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2007]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 7 is "+str(score))
    return score     
    
def lda_user2_session8():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2001, 2009, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2008]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 8 is "+str(score))
    return score 
    
def lda_user2_session9():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2001, 2010, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2009]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 9 is "+str(score))
    return score   
    
def lda_user2_session10():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2001, 2012, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 10 is "+str(score))
    return score     
    
def lda_user2_session12():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2001, 2013,
       2028, 2029, 2031, 2032] 
    test_ids = [2012]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 12 is "+str(score))
    return score 
    
def lda_user2_session13():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2032] 
    test_ids = [2013]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 13 is "+str(score))
    return score   
    
def lda_user2_session28():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2013, 2029, 2031, 2032] 
    test_ids = [2028]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 28 is "+str(score))
    return score  
    
def lda_user2_session29():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2013, 2031, 2032] 
    test_ids = [2029]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 29 is "+str(score))
    return score     
    
def lda_user2_session31():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2013, 2032] 
    test_ids = [2031]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 31 is "+str(score))
    return score  
    
def lda_user2_session32():
    train_ids = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2012, 2001,
       2028, 2029, 2031, 2013] 
    test_ids = [2032]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #rint("Accuracy for session 32 is "+str(score))
    return score 
    
def lda_user4_session1():
    train_ids = [4002]
    test_ids = [4001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 1 is "+str(score))
    return score   
    
def lda_user4_session2():
    train_ids = [4001]
    test_ids = [4002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user7_session1():
    train_ids = [7002]
    test_ids = [7001]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 1 is "+str(score))
    return score 
    
def lda_user7_session2():
    train_ids = [7001]
    test_ids = [7002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 2 is "+str(score))
    return score 
    
def lda_user8_session2():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8002]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 2 is "+str(score))
    return score   
    
def lda_user8_session3():
    train_ids = [8002, 8010, 8016, 8017, 8018, 8019]
    test_ids = [8003]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 3 is "+str(score))
    return score  
    
def lda_user8_session10():
    train_ids = [8003, 8002, 8016, 8017, 8018, 8019]
    test_ids = [8010]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 10 is "+str(score))
    return score   
    
def lda_user8_session16():
    train_ids = [8003, 8010, 8002, 8017, 8018, 8019]
    test_ids = [8016]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 16 is "+str(score))
    return score    
    
def lda_user8_session17():
    train_ids = [8003, 8010, 8016, 8002, 8018, 8019]
    test_ids = [8017]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 17 is "+str(score))
    return score  
    
def lda_user8_session18():
    train_ids = [8003, 8010, 8016, 8017, 8002, 8019]
    test_ids = [8018]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 18 is "+str(score))
    return score     
    
def lda_user8_session19():
    train_ids = [8003, 8010, 8016, 8017, 8018, 8002]
    test_ids = [8019]
    X_train, X_test, y_train, y_test = split_data_session(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    #print("Accuracy for session 19 is "+str(score))
    return score 

In [18]:
def lda_all():
    print ("User 1 : " )
    a = lda_user1_session1()
    b = lda_user1_session2()
    c = lda_user1_session3()
    print ("User 2 : " )
    d = lda_user2_session1()
    e = lda_user2_session3()
    f = lda_user2_session4()
    g = lda_user2_session5()
    h = lda_user2_session6()
    i = lda_user2_session7()
    j = lda_user2_session8()
    k = lda_user2_session9()
    l = lda_user2_session10()
    m = lda_user2_session12()
    n = lda_user2_session13()
    o = lda_user2_session28()
    p = lda_user2_session29()
    q = lda_user2_session31()
    r = lda_user2_session32()
    print ("User 4 : " )
    s = lda_user4_session1()
    t = lda_user4_session2()
    print ("User 7 : " )
    u = lda_user7_session1()
    v = lda_user7_session2()
    print ("User 8 : " )
    w = lda_user8_session2()
    x = lda_user8_session3()
    y = lda_user8_session10()
    z = lda_user8_session16()
    aa = lda_user8_session17()
    bb = lda_user8_session18()
    cc = lda_user8_session19()
    scores = np.array([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,p,q,r,s,t,u,v,w,x,y,z,aa,bb,cc])
    return scores

In [21]:
def multiple():
    scores = np.array([])
    for i in range (21):
        test = lda_all()
        scores = np.concatenate([scores,test],axis=0)
    print("Durchschnitt:")
    print((scores.mean()))
    print(scores.std())
    

mupltiple()

In [22]:
multiple()

User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
User 1 : 
User 2 : 
User 4 : 
User 7 : 
User 8 : 
